In [2]:
import os
import random
from datetime import date
from typing import cast
from shapely.geometry import Point  # Ensure you have this import if you're using Point

import geopandas as gpd
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygris
from census import Census
from IPython.display import Image
from matplotlib.patches import ConnectionPatch
from PIL import Image
from pygris import (block_groups, counties, primary_secondary_roads,
                    school_districts, tracts)
from pygris.geocode import geocode
from shapely.geometry import Point, mapping

from epymorph import geo_library
from epymorph.data_shape import Shapes
from epymorph.error import GeoValidationException
from epymorph.geo.cache import load_from_cache, save_to_cache
from epymorph.geo.static import StaticGeo

In [4]:
########################################################## Enduser Input #################################################################
STATE = input('Enter a State: ')
COUNTY = input('Enter a County: ')
YEAR = int(input('Enter a Year: '))
crs_to_use = 'EPSG:4269' 


dir = os.path.expanduser('~/Desktop/Github/EpiMoRPH-Modeling/social-determinate-of-health/scratch/Movement_model')
powerpoint = os.path.expanduser('~/Desktop/Github/EpiMoRPH-Modeling/social-determinate-of-health/scratch/powerpoints/School_movement_01-22-24')

census = Census('98a3a86a5708b65b92a50cc76d8d67280efbeb5d')

CentroidDType = np.dtype([('longitude', np.float64), ('latitude', np.float64)])

census_block_groups = block_groups(state=STATE, county=COUNTY, year=YEAR, cache=False)

census_block_groups = census_block_groups.to_crs(crs_to_use)

state_fips_list = census_block_groups['STATEFP'].unique()
county_fips_list = census_block_groups['COUNTYFP'].unique()

state_fips = ','.join(map(str, state_fips_list))
county_fips = ','.join(map(str, county_fips_list))
census_block_groups = census_block_groups[['GEOID', 'geometry', 'ALAND']]
CNTY = state_fips + county_fips

Using FIPS code '04' for input 'AZ'
Using FIPS code '013' for input 'Maricopa'


In [7]:
import os
import random
from datetime import date
from typing import cast
from shapely.geometry import Point  

import geopandas as gpd
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygris
from census import Census
from IPython.display import Image
from matplotlib.patches import ConnectionPatch
from PIL import Image
from pygris import (block_groups, counties, primary_secondary_roads,
                    school_districts, tracts)
from pygris.geocode import geocode
from shapely.geometry import Point, mapping
########################################################## Enduser Input #################################################################
STATE = input('Enter a State: ')
COUNTY = input('Enter a County: ')
YEAR = int(input('Enter a Year: '))
crs_to_use = 'EPSG:4269' 


dir = os.path.expanduser('~/Desktop/Github/EpiMoRPH-Modeling/social-determinate-of-health/scratch/Movement_model')
powerpoint = os.path.expanduser('~/Desktop/Github/EpiMoRPH-Modeling/social-determinate-of-health/scratch/powerpoints/School_movement_01-22-24')

census = Census('98a3a86a5708b65b92a50cc76d8d67280efbeb5d')

CentroidDType = np.dtype([('longitude', np.float64), ('latitude', np.float64)])

census_block_groups = block_groups(state=STATE, county=COUNTY, year=YEAR, cache=False)

census_block_groups = census_block_groups.to_crs(crs_to_use)

state_fips_list = census_block_groups['STATEFP'].unique()
county_fips_list = census_block_groups['COUNTYFP'].unique()

state_fips = ','.join(map(str, state_fips_list))
county_fips = ','.join(map(str, county_fips_list))
census_block_groups = census_block_groups[['GEOID', 'geometry', 'ALAND']]
CNTY = state_fips + county_fips

####################################################### Census #########################################################
AGE_VARS = [
    # School Age Population (Male): B01001_003-B01001_007
    "B01001_003E",  # Population (Male) 0-4 years
    "B01001_004E",  # Population (Male) 5-9 years
    "B01001_005E",  # Population (Male) 10-14 years
    "B01001_006E",  # Population (Male) 15-17 years
    "B01001_007E",  # Population (Male) 18-19 years
    # Adult Population (Male): B01001_008-B01001_016
    "B01001_008E",  # Population (Male) 20 years
    "B01001_009E",  # Population (Male) 21 years
    "B01001_010E",  # Population (Male) 22-24 years
    "B01001_011E",  # Population (Male) 25-29 years
    "B01001_012E",  # Population (Male) 30-34 years
    "B01001_013E",  # Population (Male) 35-39 years
    "B01001_014E",  # Population (Male) 40-44 years
    "B01001_015E",  # Population (Male) 45-49 years
    "B01001_016E",  # Population (Male) 50-54 years
    # Elder Population (Male): B01001_017-B01001_025
    "B01001_017E",  # Population (Male) 55-59 years
    "B01001_018E",  # Population (Male) 60-61 years
    "B01001_019E",  # Population (Male) 62-64 years
    "B01001_020E",  # Population (Male) 65-66 years
    "B01001_021E",  # Population (Male) 67-69 years
    "B01001_022E",  # Population (Male) 70-74 years
    "B01001_023E",  # Population (Male) 75-79 years
    "B01001_024E",  # Population (Male) 80-84 years
    "B01001_025E",  # Population (Male) 85+ years
    # School Age Population (Female): B01001_027-B01001_031
    "B01001_027E",  # Population (Female) 0-4 years
    "B01001_028E",  # Population (Female) 5-9 years
    "B01001_029E",  # Population (Female) 10-14 years
    "B01001_030E",  # Population (Female) 15-17 years
    "B01001_031E",  # Population (Female) 18-19 years
    # Adult Population (Female): B01001_032-B01001_040
    "B01001_032E",  # Population (Female) 20 years
    "B01001_033E",  # Population (Female) 21 years
    "B01001_034E",  # Population (Female) 22-24 years
    "B01001_035E",  # Population (Female) 25-29 years
    "B01001_036E",  # Population (Female) 30-34 years
    "B01001_037E",  # Population (Female) 35-39 years
    "B01001_038E",  # Population (Female) 40-44 years
    "B01001_039E",  # Population (Female) 45-49 years
    "B01001_040E",  # Population (Female) 50-54 years
    # Elder Population (Female): B01001_041-B01001_049
    "B01001_041E",  # Population (Female) 55-59 years
    "B01001_042E",  # Population (Female) 60-61 years
    "B01001_043E",  # Population (Female) 62-64 years
    "B01001_044E",  # Population (Female) 65-66 years
    "B01001_045E",  # Population (Female) 67-69 years
    "B01001_046E",  # Population (Female) 70-74 years
    "B01001_047E",  # Population (Female) 75-79 years
    "B01001_048E",  # Population (Female) 80-84 years
    "B01001_049E"   # Population (Female) 85+ years
]
SCHOOL_VARS = [
    "B14007_002E",  # Enrolled in nursery school, preschool
    "B14007_003E",  # Enrolled in kindergarten
    "B14007_004E",  # Enrolled in grade 1
    "B14007_005E",  # Enrolled in grade 2
    "B14007_006E",  # Enrolled in grade 3
    "B14007_007E",  # Enrolled in grade 4
    "B14007_008E",  # Enrolled in grade 5
    "B14007_009E",  # Enrolled in grade 6
    "B14007_010E",  # Enrolled in grade 7
    "B14007_011E",  # Enrolled in grade 8
    "B14007_012E",  # Enrolled in grade 9
    "B14007_013E",  # Enrolled in grade 10
    "B14007_014E",  # Enrolled in grade 11
    "B14007_015E",  # Enrolled in grade 12
]

# CBG data
query1 = {'for': 'block group: *',
          'in': f'state: {state_fips} county: {county_fips}'}
cbgs_raw = census.acs5.get([
    "B01003_001E",  # Total Population
    "B01002_001E",  # Median Age
    "B19013_001E",  # Median Household Income, infl.adj. Dollars, Past 12 Months
    "B19025_001E",  # Aggregate Household Income, infl.adj. Dollars, Past 12 Months
    "B25010_001E"   # Average Household Size

], query1, year=YEAR)

cbgs_errollemt_raw = census.acs5.get(SCHOOL_VARS, query1, year=YEAR)
cbgs_age_raw = census.acs5.get(AGE_VARS, query1, year=YEAR)

cbgs_geog = pygris.block_groups(
    state=state_fips,
    county=county_fips,
    year=YEAR,
    cache=True
)

cbgs = pd.DataFrame.from_records(cbgs_raw)
cbgs.fillna(0, inplace=True)
cbgs.replace(-666666666, 0, inplace=True)

tract_geoids = cbgs['state'] + cbgs['county'] + cbgs['tract']
geoids = tract_geoids + cbgs['block group']

cbgs = pd.DataFrame({
    'geoid': geoids,
    'population': cbgs['B01003_001E'].astype(np.int64),
    'median_age': cbgs['B01002_001E'].astype(np.float64),
    'median_income': cbgs['B19013_001E'].astype(np.int64),
    'total_income': cbgs['B19025_001E'].astype(np.int64),
    'average_household_size': cbgs['B25010_001E'].astype(np.float64)
})


# Merge geo info
cbgs = cbgs.merge(pd.DataFrame({
    'geoid': cbgs_geog['GEOID'],
    'centroid': cbgs_geog['geometry'].apply(lambda row: row.centroid.coords[0]),
    # TIGER areas are in m^2; divide by 1e6 to get km^2
    'area': cbgs_geog.ALAND / 1e6
}), on='geoid')

cbgs.insert(8, 'pop_density_km2', cbgs['population'] / cbgs['area'])

cbgs.sort_values(by='geoid', inplace=True)
cbgs.reset_index(drop=True, inplace=True)

# Drop the same CBGs as above using an inner merge
cbgs_age = pd.DataFrame.from_records(cbgs_age_raw)
cbgs_age.astype(np.int64)

geoids = cbgs_age['state'] + cbgs_age['county'] + \
    cbgs_age['tract'] + cbgs_age['block group']
cbgs_age.insert(len(cbgs_age.columns), 'geoid', geoids)
cbgs_age = cbgs[['geoid']].merge(cbgs_age, how='inner', on='geoid')
cbgs_age.reset_index(drop=True, inplace=True)
cbgs_age.drop(columns=['geoid'], inplace=True)

def bracketize(brackets: dict[str, int]) -> pd.DataFrame:
    """Using the brackets info sum the age groups for both male and female."""
    sex_offset = sum(brackets.values())

    ranges = dict[str, tuple[slice, slice]]()
    age_offset = 0
    for key, count in brackets.items():
        i_m = age_offset
        i_f = age_offset + sex_offset
        ranges[key] = slice(i_m, i_m + count), slice(i_f, i_f + count)
        age_offset += count

    return pd.DataFrame({
        key: (cbgs_age.iloc[:, slice_m].sum(axis=1) +
              cbgs_age.iloc[:, slice_f].sum(axis=1))
        for key, (slice_m, slice_f) in ranges.items()
    }, dtype=np.int64)

cbgs_age_2 = bracketize({
    '00-19': 5,
    '20-54': 9,
    '55+': 9,
})

cbgs_errollemt = pd.DataFrame.from_records(cbgs_errollemt_raw)
cbgs_errollemt.astype(np.int64)


geoids = cbgs_errollemt['state'] + cbgs_errollemt['county'] + \
    cbgs_errollemt['tract'] + cbgs_errollemt['block group']
cbgs_errollemt.insert(len(cbgs_errollemt.columns), 'geoid', geoids)

cbgs_errollemt = cbgs[['geoid']].merge(cbgs_errollemt, how='inner', on='geoid')
cbgs_errollemt.reset_index(drop=True, inplace=True)
cbgs_errollemt.drop(columns=['geoid'], inplace=True)

def enrollment_bracketize(cbgs_enrollment: pd.DataFrame) -> pd.DataFrame:
    """Using the enrollment info sum the age groups for elementary, middle, and high school."""

    brackets = {
        "Preschool": ("B14007_002E", "B14007_002E"),  # Preschool
        # Kindergarten to Grade 5
        "Elementary": ("B14007_003E", "B14007_008E"),
        "Middle": ("B14007_009E", "B14007_011E"),      # Grade 6 to Grade 8
        "High": ("B14007_012E", "B14007_015E")         # Grade 9 to Grade 12
    }

    school_ranges = {}
    for level, (start_var, end_var) in brackets.items():
        start_index = SCHOOL_VARS.index(start_var)
        end_index = SCHOOL_VARS.index(end_var) + 1
        school_ranges[level] = (start_index, end_index)

    return pd.DataFrame({
        level: cbgs_enrollment.iloc[:, start:end].sum(axis=1)
        for level, (start, end) in school_ranges.items()
    }, dtype=np.int64)


enrollment_summary = enrollment_bracketize(cbgs_errollemt)

data = {
    'label': cbgs['geoid'].to_numpy(dtype=np.str_),
    'centroid': cbgs['centroid'].to_numpy(dtype=CentroidDType),
    'area km^2': cbgs['area'].to_numpy(dtype=np.int64),
    'population': cbgs['population'].to_numpy(dtype=np.int64),
    'population_by_school': enrollment_summary.to_numpy(dtype=np.int64),
    'population_by_age': cbgs_age_2.to_numpy(dtype=np.int64),
    'median_age': cbgs['median_age'].to_numpy(dtype=np.float64),
    'median_income': cbgs['median_income'].to_numpy(dtype=np.int64),
    'average_household_size': cbgs['average_household_size'].to_numpy(dtype=np.float64),
    'pop_density_km2': cbgs['pop_density_km2'].to_numpy(dtype=np.float64),
}

census_df = pd.DataFrame({k: list(v) for k, v in data.items()})
census_block_groups = pd.merge(census_df, census_block_groups, left_on='label', right_on='GEOID')
census_block_groups = gpd.GeoDataFrame(census_block_groups, geometry=census_block_groups['geometry']).set_crs(epsg=4269, inplace=True)

census_df

,label,centroid,area km^2,population,population_by_school,population_by_age,median_age,median_income,average_household_size,pop_density_km2
0,040130101011,"[-111.77075643330394, 33.76924660709943]",18,1791,"[384, 140, 0, 139]","[358, 781, 652]",50.5,99219,2.33,99.319999
1,040130101012,"[-111.72000671148204, 33.7410933260103]",29,2007,"[162, 106, 0, 16]","[162, 423, 1422]",63.9,127614,2.15,68.782398
2,040130101013,"[-111.66697569766907, 33.72404113568911]",6,1264,"[0, 0, 0, 0]","[0, 22, 1242]",70.8,80742,1.83,183.890869
3,040130101021,"[-111.49136471795472, 33.71546544071483]",2368,1139,"[91, 10, 0, 48]","[58, 413, 668]",58.9,105987,2.04,0.480805
4,040130101022,"[-111.8505100649063, 33.82609833466325]",11,1487,"[131, 40, 28, 33]","[110, 284, 1093]",62.8,128523,2.20,125.420289
...,...,...,...,...,...,...,...,...,...,...
2500,040139801001,"[-112.88186166490277, 32.66947221385736]",2912,0,"[0, 0, 0, 0]","[0, 0, 0]",0.0,0,0.00,0.000000
2501,040139804001,"[-111.95879092988882, 33.298073567638134]",3,13,"[5, 0, 0, 0]","[0, 9, 4]",21.9,0,0.00,3.320144
2502,040139805001,"[-111.88633119332626, 33.24392529871578]",3,0,"[0, 0, 0, 0]","[0, 0, 0]",0.0,0,0.00,0.000000
2503,040139806001,"[-111.61909767565444, 33.33555256790642]",9,4291,"[1239, 384, 428, 175]","[1397, 2251, 643]",34.1,136875,3.33,432.887419
